In [1]:
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *

In [2]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [3]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a4f0c4a-0d0c-4201-ac6c-19756581a39d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 71ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [4]:
loader = IncrementalLoader(spark, '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma', 'delta_news/sports')
df = loader.get_new_data()

25/05/02 18:04:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CDF not available — doing full load


25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 1 with a timestamp 1745921639653 which is greater than the next commit timestamp 1745921639219.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639654 which is greater than the next commit timestamp 1745921639603.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639655 which is greater than the next commit timestamp 1745921639603.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 4 with a timestamp 1745921639656 which is greater than the next commit timestamp 1745921639653.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639657 which is greater than the next commit timestamp 1745921639445.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639658 which is greater than the next commit timestamp 1745921639603.
25/05/02 18:04:41 WARN DeltaHistoryManager: Fo

In [22]:
processor = NewsProcessor(spark, df)

In [23]:
processor.remove_clear_duplicates()
processor.name_to_id()
processor.ensure_schema()
processor.remove_hidden_duplicates(['url'], ['publishedAt'])

Removed 46 simple duplicate(s)
Removed 1 hidden duplicate(s)


In [24]:
processor.df.orderBy('PublishedAt', ascending=False)

url,author,content,description,publishedAt,source,title,urlToImage
https://www.axios...,Cuneyt Dil,D.C. and the Wash...,The deal would br...,2025-04-28 10:50:05,"{axios, Axios}",Scoop: Commanders...,https://images.ax...
https://www.cbssp...,,MILWAUKEE -- A hu...,Bucks coach Doc R...,2025-04-28 08:32:00,"{cbs-sports, CBS ...",Damian Lillard in...,https://sportshub...
https://lakingsin...,Zach Dooley For L...,The LA Kings were...,The LA Kings were...,2025-04-28 08:16:08,{lakingsinsider.c...,Game 4 FINAL – Ki...,https://lakingsin...
https://www.mlbtr...,Mark Polishuk,Daulton Varsho wi...,Toronto's Gold Gl...,2025-04-28 05:18:53,{mlb-trade-rumors...,Blue Jays Notes: ...,https://cdn.mlbtr...
https://apnews.co...,Tim Reynolds,Detroits Tim Hard...,The NBA acknowled...,2025-04-28 05:12:00,{associated-press...,NBA acknowledges ...,https://dims.apne...
https://www.foxsp...,Bob Pockrass,"TALLADEGA, Ala. T...",Austin Cindric wo...,2025-04-28 04:36:05,"{fox-sports, Fox ...","Kyle Larson, Denn...",https://a57.foxsp...
https://nesn.com/...,Gio Rivera,NULL,The Boston Celtic...,2025-04-28 04:23:54,"{nesn, NESN}","Four Studs, Two D...",https://s47719.pc...
https://www.dalla...,Evan Grant,SAN FRANCISCO Som...,The Rangers lost ...,2025-04-28 03:18:34,{the-dallas-morni...,Explaining the in...,https://dmn-dalla...
https://www.seatt...,Shane Lantz,Sunday couldnt ha...,Logan Evans tosse...,2025-04-28 02:48:13,{the-seattle-time...,Mariners win Loga...,https://images.se...
https://www.nbcsp...,Mike Florio,The Falcons wont ...,An email containi...,2025-04-28 02:36:07,"{nbcsports.com, N...",NFL is “reviewing...,https://nbcsports...


In [19]:
processor.df

url,author,content,description,publishedAt,source,title,urlToImage
https://nesn.com/...,Greg Dudek,NULL,Sports Illustrate...,2025-04-04 00:12:00,"{nesn, NESN}",NFL Insider Provi...,https://nesn.com/...
https://www.si.co...,Bob Harig,"DORAL, Fla. The P...",The Saudi Arabia ...,2025-04-04 04:36:53,{sports-illustrat...,Report: PGA Tour ...,https://images2.m...
https://goheels.c...,NULL,NULL,Perry Hargett's h...,2025-04-04 08:04:27,"{goheels.com, Goh...",Carolina Wins Thu...,https://images.si...
https://www.cbssp...,,SAN ANTONIO Befor...,Scheyer wanted to...,2025-04-04 08:12:00,"{cbs-sports, CBS ...",Duke's Jon Scheye...,https://sportshub...
https://www.npr.o...,Majd Al-Waheidi,The NCAA Division...,NCAA president Ch...,2025-04-04 11:00:00,"{npr, NPR}",NCAA president on...,https://npr.brigh...
https://triblive....,Tim Benz,Friday’s “First C...,Friday&rsquo;s &l...,2025-04-04 11:59:00,"{triblive, TribLIVE}",First Call: Finan...,https://assets-va...
https://www.cbsne...,NULL,"College Park, Md....",USA Fencing disqu...,2025-04-04 12:01:00,"{cbs-news, CBS News}",Fencer disqualifi...,https://assets2.c...
https://nypost.co...,Justin Terranova,If only the NCAA ...,If only the NCAA ...,2025-04-04 14:10:00,"{new-york-post, N...",The NIT champions...,https://nypost.co...
https://www.balti...,NULL,NULL,Takeaways from th...,2025-04-04 15:23:44,{baltimoreravens....,Late for Work: Ra...,https://static.cl...
https://www.nbcsp...,Josh Alper,The last two seas...,The last two seas...,2025-04-04 15:49:08,"{nbcsports.com, N...",Liam Coen: We’ve ...,https://nbcsports...
